In [1]:
print("A)")

A)


In [2]:
price = (8022+3300)+24444+999+2000+2400+3107+322+1089+1733
print(price)

47416


# overview 
In this notebook, we will implementing the K-Nearest Neighbors(KNN) algorithm from scratch. KNN is a surpervised macine learning algorithm that can be used for both classification and regression tasks. It is a non-parametric method that makes predictions based on the simillarity of new data points to the labeled data points in the training set.
 
The main objective of this notebook is to understand the inner workings of the KNN algorithm and how it can be implemented without relying on existing libraries or frameworks. By building the algorithm from scratch, we will gain a deeper understanding of its underlying principles and mechanics.

# imports

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns


In [14]:
#data analysis